In [2]:
'''set the environment'''
#import needed packages
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import fnmatch

#data path
path = ""


#create and set database
# spark.sql(f"DROP DATABASE IF EXISTS {database} CASCADE")
# spark.sql(f"CREATE DATABASE {database}")
spark.sql(f"USE {database}")

In [ ]:
schema = StructType().add("text",StringType(),True).add("label",IntegerType(),True)
df1 = spark.sql('SELECT NULL text, NULL label, NULL fileName')
count = 0

for folder in dbutils.fs.ls(path + '/training.sets.released/'):
  for file in dbutils.fs.ls(folder[0]):
    df2 = spark.read.csv(file[0], schema, sep='\u0001', quote='', header=False, lineSep='\u0002').withColumn('fileName', lit(int(file[1][:-4])))
    if count != 0:
      dfUnion = df2.union(df1)
    else:
      dfUnion = df2
    df1 = dfUnion
    count = count + 1

spark.sql('DROP TABLE IF EXISTS textTable')
df1.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('textTable')

In [ ]:
regExHintNeg = {
'(contraindication|negation|contraindicate).{0,1}'                                                                                            : 'contraindication|negation',
'(patient|pt|he|she).{0,4}? (refus.{0,2}|decline.{0,1})'                                                                                      : 'refuse|decline',
'(refus|decline).{0,18} (dvt|vte)? {0,1}(ppx|prophylaxis)'                                                                                    : 'refuse prophylaxis',
'(refus|decline).{0,5} (venous foot pumps|intermittent pneumatic compression|graduated compression stockings)'                                : 'refuse compression',
'(refus|decline).{0,5} (argatroban|dabigatran|warfarin)'                                                                                      : 'refuse argatroban|dabigatran|warfarin',
'(refus|decline).{0,5} (lovenox|enoxaparin|heparin)'                                                                                          : 'refuse lovenox|enoxaparin|heparin',
'(refus|decline).{0,5} (refludan|lepirudin|bivalirudin)'                                                                                      : 'refuse refludan|lepirudin|bivalirudin',
'(refus|decline).{0,5} (abciximab|eptifibatide|tirofiban)'                                                                                    : 'refuse abciximab|eptifibatide|tirofiban',
'(refus|decline).{0,5} (xarelto|rivaroxaban|eliquis|apixaban|arixtra|fondaparinux|savaysa|edoxaban|bevyxxa|betrixaban)'                       : 'refuse xa inhibitor|apixaban|rivaroxaban|edoxaban',
'serious interaction'                                                                                                                         : 'serious interaction',
'(did|could|)( not|n\\\'t|nt) tolerat(e|ed)'                                                                                                  : 'did not tolerate',
'no known( drug)? allergies'                                                                                                                  : 'no known drug allergies',
'(serious interaction|allerg|reaction).{0,6}'                                                                                                 : 'negative reaction in general',
'(serious interaction|allerg|reaction).{0,6} (morphine|diltiazem|penicillin|Levofloxacin|nkda)'                                               : 'negative reaction to other drugs',
'(serious interaction|reaction).{0,3} (venous foot pumps|intermittent pneumatic compression|graduated compression stockings)'                 : 'negative reaction to compression',
'(serious interaction|allerg|reaction).{0,6} (argatroban|dabigatran|warfarin)'                                                                : 'negative reaction to argatroban|dabigatran|warfarin',
'(serious interaction|allerg|reaction).{0,6} (lovenox|enoxaparin|heparin)'                                                                    : 'negative reaction to lovenox|enoxaparin|heparin',
'(serious interaction|allerg|reaction).{0,6} (refludan|lepirudin|bivalirudin)'                                                                : 'negative reaction to refludan|lepirudin|bivalirudin',
'(serious interaction|allerg|reaction).{0,6} (abciximab|eptifibatide|tirofiban)'                                                              : 'negative reaction to abciximab|eptifibatide|tirofiban',
'(serious interaction|allerg|reaction).{0,6} (xarelto|rivaroxaban|eliquis|apixaban|arixtra|fondaparinux|savaysa|edoxaban|bevyxxa|betrixaban)' : 'negative reaction to xa inhibitor|apixaban|rivaroxaban|edoxaban'
}

regExHintNegCase = ""
for item in regExHintNeg:
  regExHintNegCase += f",CASE WHEN LOWER(cnt.Text) RLIKE '\\\W{item}\\\W' THEN '{regExHintNeg[item]}' END"
regExHintNegCase = regExHintNegCase[1:]

regExHintNegCase2 = ""
for item in regExHintNeg:
  regExHintNegCase2 += f",CASE WHEN LOWER(cnt.Text) RLIKE '{item}' THEN '{regExHintNeg[item]}' END"
regExHintNegCase2 = regExHintNegCase2[1:]

regExHintNegWhere = ""
for item in regExHintNeg:
  regExHintNegWhere += f"OR LOWER(cnt.Text) RLIKE '\\\W{item}\\\W'"
regExHintNegWhere = regExHintNegWhere[3:]

In [ ]:
values = [
 (' Allergies Allergen Reactions Warfarin Other ',)
,(' Allergic to Warfarin ',)
,(' Allergies to Warfarin ',)
,(' enoxaparin ',)
,(" he couldn't tolerate ",)
,(' heparin ',)
,(' patient declined enoxaparin ',)
,(' patient declined treatment ',)
,(' patient did refuse to enoxaparin ',)
,(' patient refuses lovenox ',)
,(' pt did not tolerate ',)
,(' refusal to ppx ',)
,(' serious reaction to venous foot pumps ',)
,(' She declined routine tele and dvt ppx ',)
,(' She did decline pe ppx ',)
,(' right patella tendon. The patient tolerated the procedure well. ',)
,(' She refuses vte ppx ',)
,(' She refuses vte ppx ',)
]
someSchema = StructType([StructField("text", StringType(), True)])
spark.createDataFrame(values, someSchema).createOrReplaceTempView('text')


df = spark.sql(f"""
SELECT LOWER(text) text,
CONCAT_WS(',', {regExHintNegCase}) RegExHintNeg
FROM text cnt
WHERE {regExHintNegWhere}
""")
display(df.sort('text'))

In [ ]:

spark.sql("DROP Table IF EXISTS VTE1_NegNotes")
spark.sql(f"""
CREATE TABLE VTE1_NegNotes AS
SELECT
fileName, text, label,
CONCAT_WS(',', {regExHintNegCase}) RegExHintNeg
FROM textTable cnt
WHERE {regExHintNegWhere}
""")

In [ ]:

spark.sql("DROP Table IF EXISTS VTE1_allNotes")
spark.sql(f"""
CREATE TABLE VTE1_allNotes AS
SELECT
fileName, text, label,
CONCAT_WS(',', {regExHintNegCase}) RegExHintNeg
FROM textTable cnt
""")

In [ ]:
579350 #allergic to shrimp
236076 #does not tolerate did not tolerate BiPAP

In [ ]:
#explode the text into readable line blocks
fileName = 982259
query =  f"SELECT Text FROM VTE1_allNotes WHERE fileName = {fileName}"
explodedf = spark.sql(f"""
SELECT *,
CONCAT_WS(',', {regExHintNegCase2}) RegExHintNeg
FROM (
  SELECT EXPLODE(SPLIT(Text, '\\\n')) Text
  FROM ({query})
  ) cnt
WHERE TRIM(Text) != ''
""")
display(explodedf)

In [ ]:
manual_review = f"""
WHERE fileName NOT IN (
832713, 90588,  439437, 234430, 227793, 146925, 160647, 170332, 699107, 611947, --first data discovery, contraindication
84590, 718683, 271007, 288475, 789401, 298518, 822007, 179398, 541445, 647653, --active learning, predicted contraindication
505393, 169471, 518316, 982259, --active learning, no contraindication predict wrong
) --strong indication of contraindication, gold label
AND fileName NOT IN (
706689, 247842, 519244, 193097, 493964, 395161, 297173, 434670, 163896, 427771, --first data discovery, no contraindication
793388, 901676, 705914, 620574, 111121, 416661, 335277, 432852, 643929, 750528 --active learning, no predicted contraindication
) --strong indication of no contraindication, gold label
"""

In [ ]:
display(spark.sql(f"""
SELECT * 
FROM VTE1_NegNotes
{manual_review}
AND RegExHintNeg LIKE '%contraindication%'
"""))

In [ ]:
display(spark.sql(f"""
SELECT * 
FROM VTE1_allNotes
{manual_review}
AND RegExHintNeg NOT LIKE '%contraindication%'
"""))

In [ ]:
text = spark.sql(query).collect()[0][0]
print(text)

In [ ]:
399249,55165,428715,550263,158707,240018,410131,101994,80620,470327,277273,579350,390407,959468,14370,333151,236076,19789,510853,177325 #correct predictions of contraindication for testing
138153,272061,146692,776994,579350,790870,295864,145361,180195 #incorrect predictions of contraindication for testing

733089,116298,855014,954658,461980,959882,572498,307965,995611,610751,727615,143374,714039,106886,19768,675406,521641,921290,978936,637481 #correct predictions of no contraindication for testing
741234,801070,983755,498177,940480,943791,119940,782467,911738,952768,156475,373468,448263,894598,486416,268179,515482,258841,455105,200236,48050,387753,584367,967058,244810,92581,978936,686143 #incorrect predictions of no contraindication for testing

In [ ]:
%sql
--UPDATE textTable SET label = 'contraindication'
UPDATE textTable SET label = 1
WHERE fileName IN (
832713, 90588,  439437, 234430, 227793, 146925, 160647, 170332, 699107, 611947, --first data discovery, contraindication
84590, 718683, 271007, 288475, 789401, 298518, 822007, 179398, 541445, 647653, --active learning, contraindication
505393, 169471, 518316, 982259, --active learning, no contraindication predicted wrong

399249,55165,428715,550263,158707,240018,410131,101994,80620,470327,277273,579350,390407,959468,14370,333151,236076,19789,510853,177325, --correct predicted contraindication for testing
138153,272061,146692,776994,579350,790870,295864,145361,180195 --incorrect predictions of contraindication for testing
)

In [ ]:
%sql
--UPDATE textTable SET label = 'no contraindication'
UPDATE textTable SET label = 0
WHERE fileName IN (
706689, 247842, 519244, 193097, 493964, 395161, 297173, 434670, 163896, 427771, --first data discovery, no contraindication
793388, 901676, 705914, 620574, 111121, 416661, 335277, 432852, 643929, 750528, --active learning, no contraindication

733089,116298,855014,954658,461980,959882,572498,307965,995611,610751,727615,143374,714039,106886,19768,675406,521641,921290,978936,637481, --correct predicted no contraindication
741234,801070,983755,498177,940480,943791,119940,782467,911738,952768,156475,373468,448263,894598,486416,268179,515482,258841,455105,200236,48050,387753,584367,967058,244810,92581,978936,686143 --incorrect predicted no contraindication
) --strong indication of no contraindication

In [ ]:
%sql
UPDATE textTable SET label = ''
WHERE label IS NULL